<img width="8%" alt="TikTok.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/TikTok.png" style="border-radius: 15%">

# TikTok - Get trending songs by keyword

**Tags:** #tiktok #content #trends #songs

**Author:** [Alex Nodeland](https://www.linkedin.com/in/alexnodeland/)

**Last update:** 2024-06-20 (Created: 2024-06-20)

**Description:** This notebook demonstrates how to extract the top trending songs on top trending platform, TikTok, by keyword.

**References:**
- [Naas Documentation](https://site.naas.ai/)
- [Apify TikTok Scraper Documentation](https://apify.com/clockworks/tiktok-scraper)

## Input

### Import libraries

In [70]:
from datetime import datetime
import nest_asyncio
import pandas as pd
try:
    from apify_client import ApifyClient
except:
    !pip install --user apify-client
    from apify_client import ApifyClient

In [71]:
nest_asyncio.apply()

### Setup variables


**Mandatory**
- `APIFY_API_TOKEN`: This variable represents the API token required to access the Apify TikTok Scraper API. You can obtain this token by signing up on the Apify platform.
- `searchQueries`: This variable represents the keywords for which you want to extract the top trending songs on TikTok.


**Optional**

- `hashtags`: This variable represents the hashtags you want to search for in the TikTok videos. If not provided, the default value is `None`.
- `resultsPerPage`: This variable represents the number of results per page. If not provided, the default value is `100`.

#### Apify API Token

In [73]:
# Mandatory
APIFY_API_TOKEN = 'apify_api_........'
searchQueries = ['afrobeat']

# Optional - modify if desired
hashtags = []
resultsPerPage = 5

# Prepare the actor input
RUN_INPUT = {
    "hashtags": hashtags,
    "resultsPerPage": resultsPerPage,
    "searchQueries": searchQueries
}

#### Create connection object

In [72]:
client = ApifyClient(APIFY_API_TOKEN)

## Model

### Get data from Apify TikTok Scraper API

In [74]:
def get_songs_by_keyword(run_input) -> pd.DataFrame:
    """
    Get songs by keyword

    Args:
        keyword (str): keyword to search for

    Returns:
        response (pd.DataFrame): DataFrame with songs
    """
    print(f"Searching for songs with keyword: {run_input['searchQueries']}")
    
    try:
        # Run the Actor and wait for it to finish
        run = client.actor("clockworks/tiktok-scraper").call(run_input=run_input)
        print(f"Actor run completed successfully, with id: {run['id']}")

        # Fetch Actor results from the run's dataset
        items = [item for item in client.dataset(run["defaultDatasetId"]).iterate_items()]

        # Convert the list of items to a DataFrame
        response = pd.DataFrame(items)
    except Exception as e:
        print(f"An error occurred while running the actor: {e}")
        response = pd.DataFrame()

    return response

### Extract data from the response

In [75]:
# Instantiate the ABI Content Table Data Model as a pandas dataframe
attributes = {
    "ENTITY": [],
    "SCENARIO": [],
    "SOURCE": [],
    "PUBLISHED_DATE": [],
    "DATE & TIME": [],
    "ID": [],
    "TITLE": [],
    "TEXT": [],
    "CONCEPT": [],
    "SENTIMENT": [],
    "TARGET": [],
    "OBJECTIVE": [],
    "VIEWS": [],
    "LIKES": [],
    "COMMENTS": [],
    "SHARES": [],
    "ENGAGEMENTS": [],
    "ENGAGEMENT_SCORE": [],
    "TYPE": [],
    "AUTHOR_NAME": [],
    "AUTHOR_URL": [],
    "LENGTH": [],
    "PEOPLE_MENTIONED": [],
    "ORGANIZATION_MENTIONED": [],
    "CONTENT_TITLE_SHARED": [],
    "CONTENT_URL_SHARED": [],
    "LINKEDIN_LINKS": [],
    "IMAGE_SHARED": [],
    "TAGS": [],
    "URL": [],
    "DATE_EXTRACT": [],
    "SCENARIO_ORDER": []
}
content_template = pd.DataFrame(attributes)

def extract_data(response: pd.DataFrame, content_template: pd.DataFrame) -> pd.DataFrame:
    """
    Extract data from response

    Args:
        response (pd.DataFrame): DataFrame with songs

    Returns:
        content (pd.DataFrame): DataFrame with extracted data
    """
    print(f"Extracting data from response")
    content = content_template.copy()
    try:
        content = pd.concat([content, response.apply(lambda row: pd.Series({
            "ENTITY": row['musicMeta']['musicName'],
            "SCENARIO": f"Trending {row['searchQuery']} Songs on TikTok",
            "SOURCE": "TikTok",
            "PUBLISHED_DATE": row['createTimeISO'],
            "DATE & TIME": datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z"),
            "ID": row['id'],
            "TITLE": row['musicMeta']['musicName'],
            "TEXT": row['text'],
            "CONCEPT": "",
            "SENTIMENT": "",
            "TARGET": "",
            "OBJECTIVE": "",
            "VIEWS": row['playCount'],
            "LIKES": row['diggCount'],
            "COMMENTS": row['commentCount'],
            "SHARES": row['shareCount'],
            "ENGAGEMENTS": row['diggCount'] + row['commentCount'] + row['shareCount'] + row['collectCount'],
            "ENGAGEMENT_SCORE": "",
            "TYPE": "video",
            "AUTHOR_NAME": row['authorMeta']['nickName'],
            "AUTHOR_URL": f"https://www.tiktok.com/@{row['authorMeta']['name']}",
            "LENGTH": row['videoMeta']['duration'],
            "PEOPLE_MENTIONED": ", ".join(row['mentions']) if 'mentions' in row else '',
            "ORGANIZATION_MENTIONED": "",
            "CONTENT_TITLE_SHARED": "",
            "CONTENT_URL_SHARED": "",
            "LINKEDIN_LINKS": "",
            "IMAGE_SHARED": row['videoMeta']['coverUrl'],
            "TAGS": ", ".join([tag['name'] for tag in row['hashtags']]) if 'hashtags' in row else '',
            "URL": row['webVideoUrl'],
            "DATE_EXTRACT": datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z"),
            "SCENARIO_ORDER": ""
        }), axis=1)], ignore_index=True)
        print(f"Data extracted successfully")
    except Exception as e:
        print(f"An error occurred while extracting data: {e}")
    return content

## Output

### Call the function

In [76]:
print(f"The input is: \n", RUN_INPUT)
response = get_songs_by_keyword(RUN_INPUT)

The input is: 
 {'hashtags': [], 'resultsPerPage': 5, 'searchQueries': ['afrobeat']}
Searching for songs with keyword: ['afrobeat']
Actor run completed successfully, with id: yw2K9hdwSjKboHZhr


### Extract the output from the response

In [77]:
content = extract_data(response, content_template)
# Save the extracted data to a CSV file
content.to_csv('trending_songs.csv', index=False)
content

Extracting data from response
Data extracted successfully


,ENTITY,SCENARIO,SOURCE,PUBLISHED_DATE,DATE & TIME,ID,TITLE,TEXT,CONCEPT,SENTIMENT,...,PEOPLE_MENTIONED,ORGANIZATION_MENTIONED,CONTENT_TITLE_SHARED,CONTENT_URL_SHARED,LINKEDIN_LINKS,IMAGE_SHARED,TAGS,URL,DATE_EXTRACT,SCENARIO_ORDER
0,son original,Trending afrobeat Songs on TikTok,TikTok,2023-03-09T17:43:04.000Z,2024-06-20T15:32:29.000Z,7208603444557450501,son original,Yeah i need a MOTIVE #fyp #afrobeats #trending...,,,...,,,,,,https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,"fyp, afrobeats, trending, h4ze",https://www.tiktok.com/@itzh4ze/video/72086034...,2024-06-20T15:32:29.000Z,
1,original sound,Trending afrobeat Songs on TikTok,TikTok,2023-07-11T16:07:57.000Z,2024-06-20T15:32:29.000Z,7254593493585104155,original sound,The vibe is just right 👌🏼🔥 @IFETE ⏳ #afrobeats...,,,...,@IFETE,,,,,https://p16-sign-useast2a.tiktokcdn.com/obj/to...,", afrobeats, afrobeat, afrodance, afrodancers",https://www.tiktok.com/@afrodance.1/video/7254...,2024-06-20T15:32:29.000Z,
2,DJ Phaphane Mashup Afrobeats 2022,Trending afrobeat Songs on TikTok,TikTok,2023-12-05T23:01:28.000Z,2024-06-20T15:32:29.000Z,7309249546570288417,DJ Phaphane Mashup Afrobeats 2022,If you got them all we are offically friends🤝💙...,,,...,@DJ,,,,,https://p16-sign-useast2a.tiktokcdn.com/obj/to...,"afrobeatsdancechallenge, dance, foryou, afroda...",https://www.tiktok.com/@jacobworth01/video/730...,2024-06-20T15:32:29.000Z,
3,Sip (Alcohol),Trending afrobeat Songs on TikTok,TikTok,2023-03-19T10:33:07.000Z,2024-06-20T15:32:29.000Z,7212203499260529926,Sip (Alcohol),#joeboy #sip #alcohol #songlyrics #afrobeats #...,,,...,,,,,,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,"joeboy, sip, alcohol, songlyrics, afrobeats, k...",https://www.tiktok.com/@kingflickr/video/72122...,2024-06-20T15:32:29.000Z,
4,original sound,Trending afrobeat Songs on TikTok,TikTok,2024-02-04T13:47:49.000Z,2024-06-20T15:32:29.000Z,7331743101570108677,original sound,#ayrastarr #commas #lyricsvideo #afrobeats #af...,,,...,,,,,,https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,"ayrastarr, commas, lyricsvideo, afrobeats, afr...",https://www.tiktok.com/@lyricsvillla/video/733...,2024-06-20T15:32:29.000Z,
